### Изучение общей информации

In [1]:
#вызов библиотеки Pandas
import pandas as pd

#исследование статистики платежеспособности клиентов
statistics = pd.read_csv('/datasets/data.csv')

#Вывод первых 10 строк таблицы
print(statistics.head(5))

#просмотр сводной информации о наборе данных в таблице
print(statistics.info())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

In [2]:
statistics.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

#### Анализ первых строк таблицы с данными выявил следующее:
1) в колонке "days_employed" присутствуют значения с дробной частью. Необходимо заменить значние на целое число;
2) в колонке "education" присутствуют значения, записанные разным регистром. Необходимо привести текстовые значения к единому стандарту (регистру) вывода.
3) в колонке "children" присутствуют отрицательные значения. Необходимо заменить значение на положительное число.
4) возможно в таблице присутствуют дублирующие значения. Необходимо сделать проверку на поиск дубликаторов по всем колонкам таблицы. В случае выявления дубликатов - удалить их.

#### Анализ сводной информации о наборе данных в таблице выявил следующее:
1) таблица содержит 12 колонок и 21525 строк;
2) таблица содержит 5 колонок с с типом object (строка);
3) таблица содержит 2 колонки с типом float64 (вещественные числа);
4) таблица содержит 5 колонок с типом int64 (целые числа);
5) колонки "days_employed" и "total_income" содержат пропущенные значения; 
  5.1. Колонка "Трудовой стаж в днях" содержит пропцщенные значения возможно просто из-за ошибки заполнения. Чтобы не потерять важные данные пропуски необходимо заменить нулями;
  5.2. Колонка "Ежемесячный доход" содержит пропущенные значения возможно из-за того, что человек работает неофициально, без оформления документов. Необходимо проанализировать данные в разрезе типа занятости и заполнить пропуски медианой в разрезе каждого типа деятельности;
  5.3. Необходимо определить тип пропущенных значений в колонке "days_employed", заполнить пропуски значением "0";
  5.4. Для колонки "total_income" пропуски заменить значением медианы, вычисленной для каждого значения колонки "Тип занятости".
6) колонка "days_employed" с типом данных float64 содержит данные о трудовом стаже в днях. Данный показатель свидетельствует, что значения должны быть целочисленными. Необходимо провести замену типа данных с типа float64 на тип int64;
7) колонка "purpose" содержит сведения о целях получения кредита. Значения данной колонки могут содержать от 1 

### Обработка пропусков

In [3]:
# Безработные пенсионеры. Проставляем доход = 0 
#statistics.loc[(statistics['days_employed'].isna()) & (statistics['income_type']=='пенсионер'),'total_income']=statistics[(statistics['days_employed'].isna()) & (statistics['income_type']=='пенсионер')]['income_type'].fillna(0)

statistics[statistics['days_employed'].isna()]

#Замена пропусков в колонке "Трудовой стаж в днях"
statistics['days_employed'] = statistics['days_employed'].fillna(0)

for row in set(statistics['income_type']):
    #Расчет медианы колонки "Доход в месяц" по каждому типу занятости
    median = statistics[statistics['income_type'] == row]['total_income'].median()
    #Для каждого типа занятости, если найдено пропущенное значение в колонке "Доход в месяц" заменяем пропуск на значение медианы
    statistics.loc[statistics['income_type'] == row , 'total_income'] = statistics[statistics['income_type'] == row]['total_income'].fillna(median)

print(statistics.isnull().sum()) 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


#### Вывод

Колонки "Трудовой стаж в днях" и "Доход в месяц" содержали пропущенные числовые значения (NaN).
При выводе таблицы, отфильтрованной по условию "Трудовой стаж в днях" содержит пропуски ("NaN"), была выявлена закономерность что у всех отфильтрованных строк в колонке "Доход в месяц" также вместо значения указан пропуск NaN. Вероятно, что все эти пропуски не случайно и относятся они людям с типом занятости "Безработный".

Для того, чтобы не пропустить важные данные в таблице, вместо удаления строк:
1) методом fillna() пропуски в колонке "Трудовой стаж в днях" были заменены на значение "0".
2) в колонке "Доход в месяц" методом fillna() пропуски были заменены на значением медианы, вычисленной в разрезе каждого значения колонки "Тип занятости"

После применения всех вышеуказанных методов, таблица statistics заполнена без пропусков.

### Замена типа данных

In [4]:
#Замена типа данных в колонке "Трудовой стаж в днях"
statistics['days_employed'] = statistics['days_employed'].astype('int')
print(statistics['days_employed'])

0         -8437
1         -4024
2         -5623
3         -4124
4        340266
          ...  
21520     -4529
21521    343937
21522     -2113
21523     -3112
21524     -1984
Name: days_employed, Length: 21525, dtype: int64


#### Вывод

Тип колонки "Трудовой стаж в днях" до преобразования был - вещественное число, но трудовой стаж в днях относительно года не может быть дробным. После применения метода astype() (перевод типа в целое число), тип колонки "Трудовой стаж в днях" стал "целое число".

### Обработка дубликатов

In [5]:
#Приведение к нижнему регистру всех значений колонки "Образование клиента"
statistics['education'] = statistics.education.str.lower()
#Поиск дубликатов (полного соответствия значений) по всем колонкам таблицы
print('Количество дублирующих записей в таблице: ', statistics.duplicated().sum())

#Удаление дублирующих записей и  таблицы
statistics = statistics.drop_duplicates().reset_index(drop=True)
#Поиск дубликатов (полного соответствия значений) по всем колонкам таблицы
print('Количество дублирующих записей в таблице: ', statistics.duplicated().sum())

Количество дублирующих записей в таблице:  71
Количество дублирующих записей в таблице:  0


#### Вывод

В колонке "Образование клиента" все значения приведены к одному виду - нижний регистр букв.
В таблице statistics с помощью методов duplicated() и sum() найдено 71 полное совпадение (по всем колонкам таблицы).
С помощью метода drop_duplicates() из таблицы statistics удалены строки с полным совпадением данных. Вместе с повторяющимися строками удалены их индексы с помощью метода reset_index().

Таблица statistics содержит только уникальные строки по всем колонкам таблицы.


### Лемматизация

In [6]:
# Импорт библитеки pymystem3 с лемматизацией на русском языке:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

#Лемматизация значений колонки "Цель получения кредита"
def lemma_purpose(text):
    purposes = ['свадьба', 'недвижимость', 'автомобиль', 'образование', 'ремонт']
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        if lemma in ['жилье','строительство']:
            return 'недвижимость'
        if lemma in purposes:
            return lemma     
        
#Подсчет уникальных значений по каждой группе
statistics['purpose_lemma_group']=statistics['purpose'].apply(lemma_purpose)
Counter(statistics['purpose_lemma_group'])    

Counter({'недвижимость': 10204,
         'автомобиль': 4306,
         'образование': 4013,
         'свадьба': 2324,
         'ремонт': 607})

#### Вывод

Колонка "Цель получения" содержит следующий перечень основных лемм-целей получения кредита (значения указаны в порядке убывания количества повторений):
1. недвижимость - 10204 (значения недвижимость,жилье и строительство объединены в 1 значение);
2. автомобиль - 4306;
3. образование - 4013;
4. свадьба - 2324;
5. ремонт - 607.

### Категоризация данных

In [7]:
#Выделение категории "Тип занятости".  Подсчет количества в каждой группе.
def age_group_unemployed(row):
        # группа 'дети' при значении age < 18 лет
        # группа 'взрослые' при значениии age более 18 и менее 64, включая 64
        # группа 'пенсионеры' во всех остальных случаях
        age = row['dob_years']
        income_type = row['income_type']        
        if age < 18:
               return 'дети'
        if age <= 64:
            return income_type
        return 'пенсионер'


#Выделение категории "Доход в месяц". Подсчет количества в каждой группе.
statistics['age_group'] = statistics.apply(age_group_unemployed, axis=1)
print('Количество значений по типу занятости:')
print(statistics['age_group'].value_counts())

#Определяем границы группы "средний доход"
mean_min_income = statistics['total_income'].quantile(q=0.25)
mean_max_income = statistics['total_income'].quantile(q=0.75)
def income_group_unemployed(income):
        if income <= mean_min_income:
            return 'низкий доход'
        if (mean_min_income < income) & (income <= mean_max_income) :
            return 'средний доход'
        return 'высокий доход'

statistics['income_group'] = statistics['total_income'].apply(income_group_unemployed)
print('Количество значений по группам дохода в месяц:')
print(statistics['income_group'].value_counts())

#Создаем новый DataFrame
df_new = statistics.pivot_table(index=['age_group','income_group','purpose_lemma_group'],
                       columns='debt',
                       values='total_income',
                       aggfunc=['mean','median','count']).stack().reset_index()
#Ищем дубликаты
print('Количество дублирующих записей в таблице: ', df_new.duplicated().sum())
#Удаление дублирующих записей 
df_new = df_new.drop_duplicates().reset_index(drop=True)
print('Количество дублирующих записей в таблице: ', df_new.duplicated().sum())


Количество значений по типу занятости:
сотрудник          10959
компаньон           5016
пенсионер           3938
госслужащий         1434
дети                 101
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: age_group, dtype: int64
Количество значений по группам дохода в месяц:
средний доход    10726
низкий доход      5364
высокий доход     5364
Name: income_group, dtype: int64
Количество дублирующих записей в таблице:  0
Количество дублирующих записей в таблице:  0


#### Вывод

Можно выделить 5 значений (с наибольшим количеством) категории типа занятости:
1. сотрудник - 10959;
2. компаньон - 5016;
3. пенсионер - 3938;
4. госслужащий - 1434;
5. дети - 101.

Значения колонки "Доход в месяц" разделены на 3 группы значений. 
1. средний доход - 10726;
2. низкий доход - 5364;
3. высокий доход - 5364.

In [9]:
# Колонка "Количество детей в семье" содержит некорректное значение "-1". 
# Необходимо заменить значение на "0"
statistics['children'] = statistics['children'].replace(-1,0)

# Определяем общее количество клиентов с детьми
total_with_childs = statistics[statistics['children']>0]['children'].count()
# Определяем обзее количество клиентов без детей 
total_not_childs = statistics[statistics['children']==0]['children'].count()

#какой процент от всех клиентов с детьми (без детей) имел задолженность по возврату кредита
print('Доля клиентов с детьми, имеющих задолженность по возврату кредита в срок: {:.2f}'.format(statistics[(statistics['debt']==1) & (statistics['children']>0)]['children'].count()*100/total_with_childs))
print('Доля клиентов без детей, имеющих задолженность по возврату кредита в срок: {:.2f}'.format(statistics[(statistics['debt']==1) & (statistics['children']==0)]['children'].count()*100/total_not_childs))

#какой процент от всех клиентов с детьми (без детей) вернули кредит в срок
print('Доля клиентов с детьми, вернувших кредит в срок: {:.2f}'.format(statistics[(statistics['debt']==0) & (statistics['children']>0)]['children'].count()*100/total_with_childs))
print('Доля клиентов без детей, вернувших кредит в срок: {:.2f}'.format(statistics[(statistics['debt']==0) & (statistics['children']==0)]['children'].count()*100/total_not_childs))

Доля клиентов с детьми, имеющих задолженность по возврату кредита в срок: 9.25
Доля клиентов без детей, имеющих задолженность по возврату кредита в срок: 7.53
Доля клиентов с детьми, вернувших кредит в срок: 90.75
Доля клиентов без детей, вернувших кредит в срок: 92.47


#### Вывод

Клиенты без детей имеют больший процент случаев возврата кредитов в срок, нежели чем клиенты с детьми

In [10]:
#Приведение к нижнему регистру всех значений колонки "Семейное положение"
statistics['family_status'] = statistics.family_status.str.lower()

#какой процент клиентов всех типов семенойго положения допустили задолженность по возврату кредита
print(statistics[statistics['debt']==1].groupby('family_status')['family_status'].count()*100/statistics.groupby('family_status')['family_status'].count())

family_status
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
не женат / не замужем    9.750890
Name: family_status, dtype: float64


#### Вывод

Наибольшие проценты случаев задолженности по возврату кредита допустили люди с семейным положением "не женат / не замужем" - 9,75% и "гражданский брак" - 9,35%

In [11]:
# какой процент клиентов в разрезе групп дохода вернули кредит в срок
non_debt = statistics[statistics['debt']==0]
print(non_debt.groupby('income_group')['income_group'].count()*100/statistics.groupby('income_group')['income_group'].count())

income_group
высокий доход    92.859806
низкий доход     92.039523
средний доход    91.320157
Name: income_group, dtype: float64


#### Вывод

Наибольший процент случаев возврата кредита в срок допустили люди из группы с высоким доходом - 95,24%

In [12]:
# отображение цели кредита его погашения
statistics_group = statistics.pivot_table(index='purpose_lemma_group',
                       columns='debt',
                       values='age_group',
                       aggfunc=['count']).stack().reset_index()
# какой процент целей влияет на возврат кредита в срок
statistics_group[statistics_group['debt']==0].groupby('purpose_lemma_group')['count'].sum()*100/statistics_group.groupby('purpose_lemma_group')['count'].sum()

purpose_lemma_group
автомобиль      90.640966
недвижимость    92.679341
образование     90.779965
ремонт          94.233937
свадьба         91.996558
Name: count, dtype: float64

#### Вывод

Максимальный процент возвратов кредитов в срок отнесен на цель "ремонт" - 94,23%

### Общий вывод

1. Количество детей не влияет на возврат кредитов в срок, но клиенты вообще без детей выплачивают кредиты в срок чаще чем клиенты с детьми.
2. Люди с высоким доходом чаще остальных выплачивают кредиты в срок
3. Люди с семейным статусом "вдовец / вдова" реже всех допускают просрочки по возврату кредитов в срок.
4. Кредиты с целью "ремонт" лучше всего возвращаются в срок

#### Общий вывод:
Одинокие люди без детей и с высоким доходом луше всех выплачивают кредиты в срок взятые с целью "ремонт".